<a href="https://colab.research.google.com/github/aninmath/machine_learning/blob/main/Titanic_dataset_with_sklearn_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = sns.load_dataset('titanic')

In [ ]:
df.head()

In [ ]:
x = df.drop(['survived','alive'], axis=1)
y = df['survived']

In [ ]:
df.parch.value_counts()

In [ ]:
x.isnull().sum()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, chi2

In [ ]:
# Define column types

categorical_cols = ['sex', 'pclass','sibsp','parch','embarked','class','who','adult_male','embark_town','alone','deck']
numerical_cols = ['age','fare']

In [ ]:
x_train.head(1)

In [ ]:
x_train.isnull().sum()

In [ ]:

# Categorical pipeline: Impute missing, then OHE
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
 ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Numerical pipeline: Impute missing, then scale
numerical_transformer = Pipeline(steps=[('imputer',
                                         SimpleImputer(strategy='mean')),('scaler', MinMaxScaler())])

# Combine transformers
preprocessor = ColumnTransformer(transformers=[('cat', categorical_transformer, categorical_cols),
 ('num', numerical_transformer, numerical_cols)])

# Final pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),('classifier', LogisticRegression())])


In [ ]:
pipeline.fit(x_train, y_train)

In [ ]:
pipeline.score(x_test,y_test)

In [ ]:
x_train.head(1)

## **Say I want to use different imputation for different column**

In [ ]:

mean_impute_cols = ['age']
mode_impute_cols = categorical_cols


In [ ]:
x_train.isnull().sum()[x_train.isnull().sum()>0]

In [ ]:

mean_imputer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])


mode_imputer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')
     )
])



In [ ]:

# Combine all preprocessing
preprocessor = ColumnTransformer(transformers=[
    ('mean_impute', mean_imputer, numerical_cols),
    ('mode_impute', mode_imputer, categorical_cols)
])

# Final pipeline: preprocessing + scaling + model
final_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])


In [ ]:
print(final_pipeline)

In [ ]:

from sklearn import set_config
set_config(display='diagram')  # Enables visual display

final_pipeline  # Just type the pipeline object to render it


In [ ]:
final_pipeline.fit(x_train, y_train)

In [ ]:
final_pipeline.score(x_test,y_test)

In [ ]:
#accuracy

from sklearn.metrics import accuracy_score
y_pred = final_pipeline.predict(x_test)
accuracy_score(y_test, y_pred)